This notebook makes the predictions for the DistilBERT model on the Ott et al. dataset and calculates the evaluation metrics accordingly. This is the main reference: https://huggingface.co/docs/transformers/tasks/sequence_classification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets torch sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Loading the model:

In [ ]:
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    pipeline
)
from datasets import load_dataset, load_metric
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
model_name = "dbauer1860/distilbert-base-fake-hotel-review-detector"

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

# Ott et al. Dataset

In [ ]:
ott_dataset = load_dataset(
    "csv",
    data_files= "/content/drive/MyDrive/data/ott_et_al_processed.csv",
    delimiter=";"
)

In [ ]:
ott_dataset['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 1600
})

In [ ]:
predictions = classifier(ott_dataset['train']['text'], max_length=512, truncation=True)

In [ ]:
for prediction in predictions:
    print(f"Label: {prediction['label']}, Score: {prediction['score']}")

Label: REAL, Score: 0.9872697591781616
Label: REAL, Score: 0.9990527033805847
Label: REAL, Score: 0.9990529417991638
Label: REAL, Score: 0.9991476535797119
Label: REAL, Score: 0.99893718957901
Label: REAL, Score: 0.999082088470459
Label: REAL, Score: 0.9990625977516174
Label: REAL, Score: 0.9989809393882751
Label: REAL, Score: 0.9990137815475464
Label: REAL, Score: 0.9990899562835693
Label: REAL, Score: 0.9990819692611694
Label: REAL, Score: 0.9990787506103516
Label: REAL, Score: 0.9989707469940186
Label: REAL, Score: 0.9989302754402161
Label: REAL, Score: 0.9986068606376648
Label: REAL, Score: 0.9991331696510315
Label: REAL, Score: 0.9984367489814758
Label: REAL, Score: 0.9983528852462769
Label: REAL, Score: 0.9990696310997009
Label: REAL, Score: 0.9990260601043701
Label: REAL, Score: 0.9988837838172913
Label: REAL, Score: 0.9966962337493896
Label: REAL, Score: 0.9990768432617188
Label: REAL, Score: 0.9990221261978149
Label: REAL, Score: 0.9989903569221497
Label: REAL, Score: 0.998934

In [ ]:
predicted_labels = [prediction['label'] for prediction in predictions]
scores = [prediction['score'] for prediction in predictions]

In [ ]:
label_dict = {'FAKE': 0, 'REAL': 1}
predicted_labels_numeric = [label_dict[label] for label in predicted_labels]
true_labels = ott_dataset['train']['label']

In [ ]:
accuracy = accuracy_score(true_labels, predicted_labels_numeric)
precision = precision_score(true_labels, predicted_labels_numeric, average='binary')
recall = recall_score(true_labels, predicted_labels_numeric, average='binary')
f1 = f1_score(true_labels, predicted_labels_numeric, average='binary')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.50375
Precision: 0.5018844221105527
Recall: 0.99875
F1 Score: 0.6680602006688964
